In [1]:
from transformers import BertTokenizer, BertForMaskedLM, BertTokenizer, BertForSequenceClassification
import torch
def load_model(name):
    model_path = name
    model = BertForSequenceClassification.from_pretrained(
        model_path,
        output_attentions=False
    )
    return model.cuda()
tokenizer_path = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)

# models = ["bert-base-uncased"] + [f"./finetuned-scrumbled-wikitext2-3e-4/checkpoint-{i}" for i in range(80, 40000, 80)]
models = [f"./finetuned-scrumbled-wikitext2-3e-4/checkpoint-{i}" for i in range(4560 + 80, 40000, 80)]


In [2]:
from datasets import load_dataset

imdb = load_dataset("imdb")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)
tokenized_imdb = imdb.map(preprocess_function, batched=True)


from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Found cached dataset imdb (/home/sha43/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/sha43/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-220ce39307ff7007.arrow
Loading cached processed dataset at /home/sha43/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-547478c462eb5903.arrow
Loading cached processed dataset at /home/sha43/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-e28a7280cf0b9af3.arrow


In [3]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [4]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [5]:
model = load_model(models[0])

Some weights of the model checkpoint at ./finetuned-scrumbled-wikitext2-3e-4/checkpoint-4640 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

In [6]:

from torch.utils.data import DataLoader
from tqdm import tqdm
def preprocess_function(batch):
    batch = [{k:v for k, v in elem.items() if k != "text"} for elem in batch]
    return data_collator(batch)

test_loader = DataLoader(tokenized_imdb["test"], batch_size=64, shuffle=False, collate_fn=preprocess_function)

def evaluate(model, loader):
    model.eval().cuda()
    all_pred = []
    all_true = []
    with torch.no_grad():
        for batch in tqdm(loader):
            input_ids = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            labels = batch['labels'].cuda()
            output = model(input_ids, attention_mask=attention_mask, labels=labels)
            predicts = output["logits"].argmax(axis=1)
            all_pred.extend(predicts.cpu().numpy().tolist())
            all_true.extend(batch["labels"].cpu().numpy().tolist())
    return (np.array(all_pred) == np.array(all_true)).astype(float).mean()

In [7]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

open("finetune_res.csv", "w").close()

for model_name in models:
    model = load_model(model_name).cuda()

    training_args = TrainingArguments(
        output_dir=f"res",
        save_steps=100000000,
        learning_rate=3e-5,
        per_device_train_batch_size=16,
        num_train_epochs=1,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_imdb["train"],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )


    res = trainer.train()
    model = trainer.model
    del trainer
    torch.cuda.empty_cache()

    score = evaluate(model, test_loader)
    with open("finetune_res.csv", "a") as fout:
        print(model_name, score, sep="\t", file=fout)


Some weights of the model checkpoint at ./finetuned-scrumbled-wikitext2-3e-4/checkpoint-4640 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

Step,Training Loss
500,0.400500
1000,0.337000
1500,0.296900




Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 391/391 [05:49<00:00,  1.12it/s]
loading configuration file ./finetuned-scrumbled-wikitext2-3e-4/checkpoint-4720/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./finetuned-scrumbled-wikitext

Step,Training Loss
500,0.406400
1000,0.338300
1500,0.304800




Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 391/391 [06:06<00:00,  1.07it/s]
loading configuration file ./finetuned-scrumbled-wikitext2-3e-4/checkpoint-4800/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./finetuned-scrumbled-wikitext

...



Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 391/391 [06:21<00:00,  1.02it/s]
loading configuration file ./finetuned-scrumbled-wikitext2-3e-4/checkpoint-5840/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./finetuned-scrumbled-wikitext

Step,Training Loss
500,0.401800
1000,0.335300
1500,0.300600




Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 391/391 [06:34<00:00,  1.01s/it]
loading configuration file ./finetuned-scrumbled-wikitext2-3e-4/checkpoint-5920/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./finetuned-scrumbled-wikitext

Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 31.75 GiB total capacity; 11.08 GiB already allocated; 22.50 MiB free; 11.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF